# About The Dataset

- The dataset contains Food Delivery data from a restaurant in Melbourne, Australia. ​The restaurant has three branches around the CBD area. All three branches share the same menu but they have different management so they operate differently.

## Data Dictionary:

- `order_id` - A unique id for each order.
- `date` - The date the order was made, given in YYYY-MM-DD format.
- `time` - The time the order was made, given in hh:mm:ss format.
- `order_type` - A categorical attribute representing the different types of orders namely: Breakfast, Lunch or Dinner.
- `branch_code` - A categorical attribute representing the branch code inwhich the order was made. Branch information is given in the `​branches.csv` file.
- `order_items` - A list of tuples representing the order items: first element of the tuple is the item ordered, and the second element is the quantity ordered for such item.
- `order_price` - A float value representing the order total price.
- `customer_lat` - Latitude of the customer coming from the `​nodes.csv` ​ file.
- `customer_lon` - Longitude of the customer coming from the `​nodes.csv`file.
- ` customerHasloyalty?` - A logical variable denoting whether the customer has a loyalty card with the restaurant (1 if the customer has loyalty and 0 otherwise).
- `distance_to_customer_KM` - A float representing the shortest distance, in kilometers, between the branch and the customer nodes with respect to the ​nodes.csv​ and the `​edges.csv` ​ files.

​Dijkstra algorithm ​ can be used to find the shortest path between two nodes in a graph.


- `delivery_fee` - A float representing the delivery fee of the order.


## Project Goals 

- Write a python script to transform the data into a usable format for a BI Analyst.

    -  The transformation can generate more than one table if needed.

- Analyze the data.

    -  Highlight critical errors that need to be corrected before the BI analyst can make use of the data.

## Step 0: Initiate Libraries




In [1]:
# Import Warnings
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

# Import Key Libraries
import numpy as np
import pandas as pd
import os

# Import Data Preprocessing Libraries
from dateutil.parser import parse

# ast : Abstract Syntax Trees
from ast import literal_eval

# Import Geospatial Libraries
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import *
import reverse_geocoder as rg 

# Datetime
import datetime
import datetime as dt


# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

## Step 1: Read Data

- Here will we be reading the raw data as -  `dirty_data.csv` file into our jupyter notebook.
- The variable name for the Food Delivery data would be called `dataset` .

In [2]:
delivery_data = pd.read_csv('data/dirty_data.csv')

# Copy the data
orders_df = delivery_data.copy()

### Step 1.1: Data Discovery (Building Intuition)

- This is a technique we use to get an initial feel for our data tables.
- We read the data using pandas and perform method calls.
- Standardize dataset columns in the correct format.
- Explore Descriptive Statistics on Numerical Columns and more below:

##### `df.info()`

- It is an important and widely used method of Python.
- This Method prints the information or summary of the dataframe.
- It prints the various information of the Dataframe such as index type, dtype, columns, non-values, and memory usage. It gives a quick overview of the dataset.
- Info Method to get the Non-Null Count & Dtype (data type) of the dataset,
- Validate if a column and column type aligns with the format of the Business Requirements.

In [3]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


Based on this information above:

- The dataset has 500 rows and 12 columns.
- The dataset has 0 null values.


Data transformations and cleaning tasks that might be needed include:

- Checking the data type of all columns to make sure they are in the appropriate format for analysis.
- The `order_id`, `date`, `time`, `order_type`, `branch_code`, `order_items` columns datatypes need to be changed.
- The `date` &  `time` need to be merged into a single column, renamed to `order_date` and converted to `datetime` dtype format for Parsing Dates.
- Extracting the food items and their quantities from the `order_items` column into separate columns.
- Checking the category column entries for case sensitivity (Checking if the values in the branch_code and order_type columns are consistent and correctly labeled.)
- feature engineer the `order_date`  column, potentially separating them into year, month, day, and hour columns.
- Removing any unnecessary or irrelevant columns.
- Checking for outliers in numerical columns such as order_price, distance_to_customer_KM, and delivery_fee.
- Checking if the customerHasloyalty? column only contains binary values (0 and 1).
- Checking if the  `distance_to_customer_KM` column is consistent with the values in the `nodes.csv` file.
- Checking if the `delivery_fee` column is consistent with the values in the `edges.csv`  file.
- Checking if the `order_price` column is consistent with the values in the `order_items`  column.
- Checking if the `order_price` column is consistent with the values in the  `delivery_fee` column.


Remember, the goal of this process is to ensure that your dataset is clean, understandable, and ready for further Business Intelligence analysis.


### Step 1.2: Data Preporcessing - Cleaning 

- Here we will be cleaning the data by converting the columns to the correct data types.
- We will merge the date and time columns into one column called `order_date` and convert it to a datetime type.
- We will also rename the `customerHasloyalty?` , `distance_to_customer_KM` columns to `customer_loyalty` , and  `distance_to_customer` respectively.
- We will add an additional column called `updated at` which will be the date and time the data was updated.

In [4]:
def reverseGeocode(coordinates): 
    result = rg.search(coordinates)
    return (result)

def cleaning_data_types(orders_df):
    # Create a copy of the DataFrame to avoid modifying the original one
    df_clean = orders_df.copy()

    # Define helper function to clean date data
    def clean_date(date_str):
        date_str = date_str.strip()
        date = parse(date_str, dayfirst=True)  # dayfirst=True to handle DD/MM/YYYY properly
        return date.strftime('%Y-%m-%d')

    # Convert columns to appropriate data types
    df_clean['order_id'] = df_clean['order_id'].str.extract('(\d+)').astype(int)
    df_clean['date'] = df_clean['date'].apply(clean_date).astype('datetime64[ns]')
    df_clean['datetime'] = pd.to_datetime(df_clean['date'].astype(str) + ' ' + df_clean['time'])
    df_clean["order_type"] = df_clean["order_type"].astype("category")

   
    # Convert 'branch_code' to upper case to handle case-insensitive duplicates
    df_clean["branch_code"] = df_clean["branch_code"].str.upper().astype("category")


    # Use exception handling for potential errors in the literal_eval() function
    try:
        df_clean["order_items"] = df_clean["order_items"].apply(literal_eval)
    except (ValueError, SyntaxError):
        pass

    # Continue with the remaining conversions
    df_clean["order_price"] = df_clean["order_price"].astype(float)
    df_clean["customer_lat"] = df_clean["customer_lat"].astype(float)
    df_clean["customer_lon"] = df_clean["customer_lon"].astype(float)
    df_clean["customerHasloyalty?"] = df_clean["customerHasloyalty?"].astype(bool)
    df_clean["distance_to_customer_KM"] = df_clean["distance_to_customer_KM"].astype(float)
    df_clean["delivery_fee"] = df_clean["delivery_fee"].astype(float)


    # make the order_price two decimal places
    df_clean['order_price'] = df_clean['order_price'].round(2)

    # make the delivery fee two decimal places
    df_clean['delivery_fee'] = df_clean['delivery_fee'].round(2)


    # transform long/lat into state
    coordinates =list(zip(df_clean['customer_lat'],df_clean['customer_lon'])) # generates pair of (lat,long)
    data = reverseGeocode(coordinates)


    # Create a new column with the City name    
    df_clean['name'] = [i['name'] for i in data]
    df_clean['admin1'] = [i['admin1'] for i in data]
    df_clean['admin2'] = [i['admin2'] for i in data]


    df_clean.drop(['admin1', 'admin2'], axis=1, inplace=True)
    df_clean.rename(columns={'name': 'location'}, inplace=True)
 

    # Rename the customerHasloyalty? column to customerHasloyalty
    df_clean.rename(columns={'customerHasloyalty?': 'customer_loyalty'}, inplace=True)

    # Rename the distance_to_customer_KM column to distance_to_customer_km
    df_clean.rename(columns={'distance_to_customer_KM': 'distance_to_customer_km'}, inplace=True)

    # Drop the 'date' and 'time' columns
    df_clean.drop(['date', 'time'], axis=1, inplace=True)

    # Rename the 'datetime' column to 'order_date' and move it to the second position
    df_clean.rename(columns={'datetime': 'order_date'}, inplace=True)
    order_date = df_clean.pop('order_date')
    df_clean.insert(1, 'order_date', order_date)

    # Add the 'updated_at' column with the current datetime
    df_clean['updated_at'] = datetime.datetime.today().replace(second=0, microsecond=0)

    df_clean.drop(['customer_lat', 'customer_lon'], axis=1, inplace=True)

    return df_clean

df_clean = cleaning_data_types(orders_df)
df_clean.info()


Loading formatted geocoded file...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   order_id                 500 non-null    int64         
 1   order_date               500 non-null    datetime64[ns]
 2   order_type               500 non-null    category      
 3   branch_code              500 non-null    category      
 4   order_items              500 non-null    object        
 5   order_price              500 non-null    float64       
 6   customer_loyalty         500 non-null    bool          
 7   distance_to_customer_km  500 non-null    float64       
 8   delivery_fee             500 non-null    float64       
 9   location                 500 non-null    object        
 10  updated_at               500 non-null    datetime64[ns]
dtypes: bool(1), category(2), datetime64[ns](2), float64(3), int64(

In [5]:
df_clean.head(11)

,order_id,order_date,order_type,branch_code,order_items,order_price,customer_loyalty,distance_to_customer_km,delivery_fee,location,updated_at
0,1406,2018-07-08 15:16:03,Lunch,NS,"[(Fries, 6), (Salad, 4)]",140.80,True,8.335,13.70,Docklands,2023-07-06 01:30:00
1,10125,2018-12-01 08:20:16,Breakfast,NS,"[(Cereal, 8), (Pancake, 6)]",313.50,True,7.536,6.17,Melbourne,2023-07-06 01:30:00
2,4175,2018-06-07 14:05:04,Lunch,NS,"[(Steak, 3), (Salad, 1), (Chicken, 6), (Fries,...",714.00,False,9.860,15.09,Richmond,2023-07-06 01:30:00
3,3691,2018-04-26 11:43:05,Dinner,NS,"[(Pancake, 9), (Eggs, 10), (Cereal, 2)]",480.25,False,8.614,13.68,Southbank,2023-07-06 01:30:00
4,4094,2018-04-10 11:12:40,Breakfast,NS,"[(Eggs, 5), (Coffee, 3), (Pancake, 9), (Cereal...",497.75,False,8.802,13.76,Southbank,2023-07-06 01:30:00
5,10193,2018-10-15 17:27:53,Dinner,TP,"[(Fish&Chips, 5), (Shrimp, 5), (Salmon, 2), (P...",664.50,False,9.081,13.39,Kamaishi,2023-07-06 01:30:00
6,958,2018-05-25 12:43:56,Breakfast,BK,"[(Salad, 2), (Steak, 7), (Chicken, 2)]",413.40,False,6.412,12.01,Collingwood,2023-07-06 01:30:00
7,3630,2018-09-30 16:57:27,Dinner,NS,"[(Shrimp, 8), (Fish&Chips, 6), (Salmon, 4), (P...",998.50,False,7.759,16.25,Kamaishi,2023-07-06 01:30:00
8,5755,2018-07-03 10:01:41,Breakfast,NS,"[(Cereal, 1), (Coffee, 7), (Pancake, 3)]",146.25,False,8.996,14.11,East Melbourne,2023-07-06 01:30:00
9,8573,2018-04-21 11:32:57,Breakfast,NS,"[(Eggs, 7), (Cereal, 8), (Pancake, 3), (Coffee...",417.25,False,8.624,15.76,Melbourne,2023-07-06 01:30:00


In [6]:
df_clean.to_csv('clean_data.csv', index=False)

### Step 1.3: Feature Engineering

- Here we will be creating new columns from existing columns.

#### Step 1.4: Feature Engineering ( `order_items` )

- Here we will be extracting the food items and their quantities from the `order_items` column into separate columns.
- `cuisine` which will be the type of cuisine the food item is.
-  `order_items_count` which will be the total number of items ordered.
-  `order_items_total` which will be the total price of the items ordered.

#### Step 1.5: Feature Engineering ( `order_date` )

- Here we will be creating new columns based on the existing columns in the dataset.
- `order_time_of_day` which will be the hour of the day the order was made. (Morning, Afternoon, Evening, Night)
- `order_day` which will be the day of the week the order was made. (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday)
-  `order_month` which will be the month of the year the order was made. (January, February, March, April, May, June, July, August, September, October, November, December)
- `order_season` which will be the season the order was made. (Summer, Autumn, Winter, Spring)


In [7]:
def feature_en(df_clean):
    # Create a copy of the DataFrame to avoid modifying the original one
    df = df_clean.copy()

   # Explode the 'order_items' column
    df_exploded = df.explode('order_items')

    # Split the tuple into two new columns
    df_exploded[['cuisine', 'quantity_ordered']] = pd.DataFrame(df_exploded['order_items'].tolist(), index=df_exploded.index)

    # average_order_price = order_items_total / quantity_ordered
    df_exploded['average_item_price'] = df_exploded['order_price'] / df_exploded['quantity_ordered']

    # two decimal places
    df_exploded['average_item_price'] = df_exploded['average_item_price'].round(2)

    # Drop the 'order_items' column
    df_exploded.drop('order_items', axis=1, inplace=True)


    # Extract the year, month as Jan, Feb, Mar, etc. Add them as new columns
    df_exploded['order_month'] = df_exploded['order_date'].dt.strftime('%b')
    df_exploded['day_of_week'] = df_exploded['order_date'].dt.strftime('%a')

    # new column for the season the order was made. (Spring, Summer, Autumn, Winter)
    df_exploded['order_season'] = df_exploded['order_date'].dt.month.apply(lambda x: (x%12 + 3)//3)
    
    # change the season number to season name
    df_exploded['order_season'] = df_exploded['order_season'].map({1:'Spring', 2:'Summer', 3:'Autumn', 4:'Winter'})

    # reposition the columns 
    df_exploded = df_exploded[['order_id', 'order_date' , 'order_price', 'quantity_ordered' , 'average_item_price', 'order_month', 'day_of_week', 'order_season' , 'order_type', 'branch_code' , 'delivery_fee', 'location', 'cuisine' , 'customer_loyalty', 'distance_to_customer_km','updated_at']]

    return df_exploded

df_exploded = feature_en(df_clean)
df_exploded.head()

,order_id,order_date,order_price,quantity_ordered,average_item_price,order_month,day_of_week,order_season,order_type,branch_code,delivery_fee,location,cuisine,customer_loyalty,distance_to_customer_km,updated_at
0,1406,2018-07-08 15:16:03,140.8,6,23.47,Jul,Sun,Autumn,Lunch,NS,13.70,Docklands,Fries,True,8.335,2023-07-06 01:30:00
0,1406,2018-07-08 15:16:03,140.8,4,35.20,Jul,Sun,Autumn,Lunch,NS,13.70,Docklands,Salad,True,8.335,2023-07-06 01:30:00
1,10125,2018-12-01 08:20:16,313.5,8,39.19,Dec,Sat,Spring,Breakfast,NS,6.17,Melbourne,Cereal,True,7.536,2023-07-06 01:30:00
1,10125,2018-12-01 08:20:16,313.5,6,52.25,Dec,Sat,Spring,Breakfast,NS,6.17,Melbourne,Pancake,True,7.536,2023-07-06 01:30:00
2,4175,2018-06-07 14:05:04,714.0,3,238.00,Jun,Thu,Autumn,Lunch,NS,15.09,Richmond,Steak,False,9.860,2023-07-06 01:30:00


In [8]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1596 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   order_id                 1596 non-null   int64         
 1   order_date               1596 non-null   datetime64[ns]
 2   order_price              1596 non-null   float64       
 3   quantity_ordered         1596 non-null   int64         
 4   average_item_price       1596 non-null   float64       
 5   order_month              1596 non-null   object        
 6   day_of_week              1596 non-null   object        
 7   order_season             1596 non-null   object        
 8   order_type               1596 non-null   category      
 9   branch_code              1596 non-null   category      
 10  delivery_fee             1596 non-null   float64       
 11  location                 1596 non-null   object        
 12  cuisine                  1596 non-null  

### Step 2: Descriptive Statistics 

- Descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset's distribution, excluding NaN values. 

In [ ]:
# Descriptive Statistics
df_exploded.describe(include='all')

In [ ]:
# Base on the information from the describe() method:

#  Orders Date Range 
print('Orders Date Range ---->: ', df_exploded['order_date'].min(), 'to', df_exploded['order_date'].max())

# Order Price Range
print('Order Price Range ---->: ', df_exploded['order_price'].min(), 'to', df_exploded['order_price'].max())

# Average Order Price in 2 decimal places
print('Average Order Price ---->: ', round(df_exploded['order_price'].mean(), 2))

# Popular Order Type
print('Popular Order Type ---->: ', df_exploded['order_type'].mode()[0])

# Popular Cuisine
print('Popular Cuisine ---->: ', df_exploded['cuisine'].mode()[0])

# Popular Branch
print('Popular Branch ---->: ', df_exploded['branch_code'].mode()[0])

# Popular Day of the Week
print('Popular Day of the Week ---->: ', df_exploded['day_of_week'].mode()[0])

# Popular Season
print('Popular Season ---->: ', df_exploded['order_season'].mode()[0])

# Popular Month
print('Popular Month ---->: ', df_exploded['order_month'].mode()[0])

# Pupluar Hour
print('Popular Hour ---->: ', df_exploded['order_date'].dt.hour.mode()[0])

# Delivery Fee Range
print('Delivery Fee Range ---->: ', df_exploded['delivery_fee'].min(), 'to', df_exploded['delivery_fee'].max())

# Average Disance to Customer in 2 decimal places
print('Average Disance to Customer ---->: ', round(df_exploded['distance_to_customer_km'].mean(), 2))

# Popular Location
print('Popular Location ---->: ', df_exploded['location'].mode()[0])

# Types of cuisine
print('Types of cuisine ---->: ', df_exploded['cuisine'].unique())

###

### Step 1.4: Exploratory Data Analysis

- Here we will be performing an exploratory data analysis on the data to get a better understanding of the data.